In [8]:
print("ok")

ok


In [9]:
%pwd

'c:\\All folders\\Project Files\\end-to-end-medical-chatbot-GEN-AI'

In [10]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [11]:
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [12]:
extracted_data = load_pdf(data='data/')

In [13]:
len(extracted_data)

637

In [14]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [15]:
text_chunks = text_split(extracted_data=extracted_data)

In [16]:
len(text_chunks)

5860

In [17]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [18]:
def download_huggingface_embeddings():
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [19]:
embeddings = download_huggingface_embeddings()

C:\Users\chsri\AppData\Local\Temp\ipykernel_6196\3133499717.py:2: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\All folders\Project Files\end-to-end-medical-chatbot-GEN-AI\medibot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
query_result = embeddings.embed_query("hello world")
len(query_result)

384

In [87]:
from dotenv import load_dotenv
load_dotenv()

True

In [89]:
import os

PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

In [27]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os


pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medical-chatbot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

{
    "name": "medical-chatbot",
    "metric": "cosine",
    "host": "medical-chatbot-7dthmcx.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [ ]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(documents=text_chunks,
                                              embedding=embeddings,
                                              index_name=index_name)

In [29]:
docsearch = PineconeVectorStore.from_existing_index(embedding=embeddings,
                                                    index_name=index_name)

In [30]:
docsearch

In [31]:
retriever = docsearch.as_retriever(search_type='similarity', search_kwargs={'k':3})

In [32]:
result = retriever.invoke('what is acne?')

In [33]:
result

[Document(id='db0729aa-a01c-4f1f-b527-9291b0cdcb1d', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='a6b31ff9-b767-4202-ae6d-6d495e501afd', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM -

In [93]:
from langchain_openai import ChatOpenAI

OPENROUTER_API_KEY = "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"
llm = ChatOpenAI(model="mistralai/mistral-small-3.1-24b-instruct:free",  api_key=OPENROUTER_API_KEY,
                 base_url="https://openrouter.ai/api/v1",)

In [66]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you"
    "don't know. Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

In [94]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [95]:
response = rag_chain.invoke({"input": "what is Acne?"})
print(response['answer'])

Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when pores become clogged with oil, dead skin cells, and bacteria. This condition is also known medically as acne vulgaris.


In [96]:
response = rag_chain.invoke({"input": "what is stats?"})
print(response['answer'])

I don't know.


In [97]:
response = rag_chain.invoke({"input": "what is abdominal wall defects?"})
print(response['answer'])

Abdominal wall defects are congenital (birth) defects where the stomach or intestines can protrude through the abdominal wall. They are noticeable at birth, often apparent through a bulge at the base of the umbilical cord. These defects are typically treated surgically to repair the hole and reposition the organs.
